In [1]:
pip -q install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 23.1 MB/s eta 0:00:00


In [2]:
import numpy as np
import time
import os
from transformers import TFAutoModelForCausalLM, AutoTokenizer,AutoModelForCausalLM
import tensorflow as tf
from transformers.utils import logging
logging.set_verbosity_error()

In [3]:
class ChatBot():
    """
    A class representing a chatbot that uses the DialoGPT-medium model to generate responses.

    Attributes:
        chat_history_ids (tf.Tensor): Tensor containing the chat history of the conversation.
        user_input_ids (tf.Tensor): Tensor containing the user's input.
        end_chat (bool): Flag indicating if the chat should be ended.
        tokenizer (AutoTokenizer): Tokenizer for encoding and decoding text.
        model (TFAutoModelForCausalLM): DialoGPT-medium model for generating responses.

    Methods:
        welcome(): Greets the user and provides instructions on how to end the chat.
        get_user_input(): Gets user input and sets the user_input_ids attribute.
        get_bot_response(): Generates a response from the chatbot and prints it.
        get_random_response(): Generates a random response for the chatbot.
        start_chatting(): Starts the chatbot conversation.
    """

    def __init__(self):
        """
        Initializes the ChatBot instance.

        Sets the chat_history_ids and user_input_ids attributes to None, and the end_chat attribute to False.
        Initializes the tokenizer and model attributes with the DialoGPT-medium model.
        Calls the welcome method to greet the user.

        Parameters:
        None

        Returns:
        None
        """
        # Set the initial chat history and user input IDs to None, and end_chat flag to False
        self.chat_history_ids = None
        self.user_input_ids = None
        self.end_chat = False

        # Initialize the tokenizer and model with the DialoGPT-medium model
        self.tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium", padding_side='left')
        self.model = TFAutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
        # Call the welcome method to greet the user
        self.welcome()


    def welcome(self):
        """
        Prints a greeting message, informs the user how to end the chat, 
        and provides instructions on how to interact with the ChatBot.

        Returns:
        None
        """
        print("Initializing ChatBot ...")
        time.sleep(2)
        print('Type "bye" or "quit" or "exit" to end chat \n')
        time.sleep(3)
        greeting = np.random.choice([
            "Welcome, I am ChatBot, here for your kind service",
            "Hey, Great day! I am your virtual assistant",
            "Hello, it's my pleasure meeting you",
        ])
        print("ChatBot >>  " + greeting)


    def get_user_input(self):
        """
        Gets user input and sets the user_input_ids attribute.

        If the user types "bye", "quit", or "exit", sets the end_chat attribute to True.
        """
        text = input("User    >> ")
        if text.lower().strip() in ['bye', 'quit', 'exit']:
            self.end_chat = True
            print('ChatBot >>  See you soon! Bye!')

            print('\nQuitting ChatBot ...')
        else:
            self.user_input_ids = self.tokenizer.encode(text + self.tokenizer.eos_token, \
                                                        return_tensors='tf')
    
    def get_bot_response(self):
        """
        Generates a response from the chatbot and prints it.

        If there is chat history, concatenates it with the user input to generate the bot_input_ids.
        Otherwise, uses the user input ids as the bot_input_ids.
        Generates a response from the model using the bot_input_ids.
        Decodes the response using the tokenizer and sets it to the response variable.
        If the response is empty, calls the get_random_response method to generate a random response.
        Prints the response.
        Resets chat_history_ids to None after each response is generated.
        """
        if self.chat_history_ids is not None:
            # If there is chat history, concatenate it with the user input to generate bot_input_ids
            bot_input_ids = tf.concat([self.chat_history_ids, self.user_input_ids], axis=-1)
        else:
            # Otherwise, use the user input ids as the bot_input_ids
            bot_input_ids = self.user_input_ids

        # Generate a response from the model using the bot_input_ids
        self.chat_history_ids = self.model.generate(bot_input_ids, max_length=1000, \
                                                    pad_token_id=self.tokenizer.eos_token_id)

        # Decode the response using the tokenizer and set it to the response variable
        response = self.tokenizer.decode(self.chat_history_ids[:, self.user_input_ids.shape[-1]:][0], \
                                         skip_special_tokens=True)

        # If the response is empty, call the get_random_response method to generate a random response
        if response == "":
            response = self.get_random_response()

        # Print the response
        print('ChatBot >>  ' + response)

        # Reset chat_history_ids after each response is generated
        self.chat_history_ids = None


    def get_random_response(self):
        """
        Returns a random response when the bot is unable to generate a valid response.

        If the bot is unable to generate a response, this method searches through the chat history to find the most recent 
        non-empty response. It does this by iterating backwards through the chat history, starting with the most recent 
        message and ending with the earliest message. If the non-empty response ends with a question mark, the method 
        returns a random reply from the list of "I don't know" and "I am not sure". Otherwise, the method returns a 
        random reply from the list of "Great", "Fine. What's up?", and "Okay".

        Returns:
        --------
        reply : str
            A random response string.
        """
        # Start by checking the most recent response in the chat history
        i = -1
        response = self.tokenizer.decode(self.chat_history_ids[:, self.user_input_ids.shape[i]:][0], skip_special_tokens=True)
        
        # If the most recent response is empty, search backwards through chat history to find the most recent non-empty response
        while response == '':
            i = i - 1
            response = self.tokenizer.decode(self.chat_history_ids[:, self.user_input_ids.shape[i]:][0], skip_special_tokens=True)
        
        # If the non-empty response ends with a question mark, return a random reply from the list 
        if response.strip() == '?':
            reply = np.random.choice(["I don't know", "I am not sure"])
        # Otherwise, return a random reply from the list 
        else:
            reply = np.random.choice(["Great", "Fine. What's up?", "Okay"])
        
        return reply

    @classmethod
    def start_chatting(cls):
        """
        Function to start the chatbot conversation.

        The function uses a while loop to repeatedly ask the user for input,
        generates a response using the get_bot_response() function, and prints
        the response. If the user types "bye", "quit", or "exit", the loop is
        terminated and the function returns.

        Args:
            cls: Class object

        Returns:
            None
        """

        # create an instance of the class
        chatbot = cls()

        # Loop to repeatedly ask for user input and generate responses
        while True:
            # Get input from the user and store it in a variable
            user_input = chatbot.get_user_input()

            # If user input indicates end of conversation, break the loop
            if chatbot.end_chat:
                break

            # Generate and print a response from the chatbot using the user input
            chatbot.get_bot_response()

In [ ]:
ChatBot.start_chatting()

Initializing ChatBot ...
Type "bye" or "quit" or "exit" to end chat 

ChatBot >>  Hey, Great day! I am your virtual assistant
User    >> Hello 
ChatBot >>  Hello! :D
User    >> How are you today?
ChatBot >>  I'm good, how about you?
User    >> I am good thanks for asking
ChatBot >>  I'm not sure if you're being serious or not, but I'm not sure if you're being serious or not, but I'm not sure if you're being serious or not.
User    >> What would you recommend to a person who wants to start programming?
ChatBot >>  I would recommend the book Programming by Robert C. Hall.
User    >> What does Programming by Robert C. Hall cover?
ChatBot >>  I think it covers the basics of programming.
User    >> Is it important to learn the basics of programming?
ChatBot >>  It's not necessary, but it's a good way to get started.
User    >> What is the best way to stay consistent 
ChatBot >>  I'm not sure, but I think it's to keep your head up and keep your head down.
